#ToolRavenV2 INFERENCE
This notebook is taking inference of ToolRavenV2 for a given query.It includes the following:

 - Create functions for conversion of given json format of our tools to nested tool function format as required for the model and  the response generated by model to json format as required

 - Take inference of the model for a given query of your choice

### Load Tool

In [ ]:
tools = {
  "tools": [
    {
      "tool_description": "Returns a list of work items matching the request.",
      "tool_name": "works_list",
      "title": "works_list",
      "return_type": "array of objects",
      "argument_list": [
        {
          "argument_name": "applies_to_part",
          "argument_description": "Filters for work belonging to any of the provided parts",
          "argument_type": "array of strings",
          "example": [
            ["FEAT-123"],
            ["ENH-123", "PROD-123", "CAPL-123", "CAPL-359"],
            ["HGH-262", "FEAT-007"]
          ]
        },
        {
          "argument_name": "created_by",
          "argument_description": "Filters for work created by any of these users",
          "argument_type": "array of strings",
          "example": [["DEVU-123"], ["PRO-233", "CRE-1233"]]
        },
        {
          "argument_name": "issue.priority",
          "argument_description": "Filters for issues with any of the provided priorities. Allowed values: p0, p1, p2, p3.",
          "allowed_values": ["p0", "p1", "p2", "p3"],
          "argument_type": "array of strings",
          "example": [["p0"], ["p0", "p3"], ["p0", "p1", "p2", "p3"]]
        },
        {
          "argument_name": "issue.rev_orgs",
          "argument_description": "Filters for issues with any of the provided Rev organizations",
          "argument_type": "array of strings",
          "example": [["REV-123"], ["REV-468", "REV-979"]]
        },
        {
          "argument_name": "limit",
          "argument_description": "The maximum number of works to return. The default is 50",
          "argument_type": "integer(int32)",
          "example": [40, 25, 2, 1, 50]
        },
        {
          "argument_name": "owned_by",
          "argument_description": "Filters for work owned by any of these users",
          "argument_type": "array of strings",
          "example": [["DEVU-123"], ["CAPL-264", "HGH-190"]]
        },
        {
          "argument_name": "stage.name",
          "argument_description": "Filters for records in the provided stage(s) by name",
          "argument_type": "array of strings",
          "example": [["triage"], ["design", "triage"]]
        },
        {
          "argument_name": "ticket.needs_response",
          "argument_description": "Filters for tickets that need a response",
          "argument_type": "boolean",
          "example": ["True", "False"]
        },
        {
          "argument_name": "ticket.rev_org",
          "argument_description": "Filters for tickets associated with any of the provided Rev organizations",
          "argument_type": "array of strings",
          "example": [["REV-123"], ["REV-238", "REV-119"]]
        },
        {
          "argument_name": "ticket.severity",
          "argument_description": "Filters for tickets with any of the provided severities. Allowed values: blocker, high, low, medium",
          "argument_type": "array of strings",
          "allowed_values": ["blocker", "high", "low", "medium"],
          "example": [
            ["blocker"],
            ["blocker", "high"],
            ["blocker", "high", "low"]
          ]
        },
        {
          "argument_name": "ticket.source_channel",
          "argument_description": "Filters for tickets with any of the provided source channels",
          "argument_type": "array of strings",
          "example": [["slack"], ["github"], ["slack", "scrum"]]
        },
        {
          "argument_name": "type",
          "argument_description": "Filters for work of the provided types. Allowed values: issue, ticket, task",
          "allowed_values": ["issue", "ticket", "task"],
          "argument_type": "array of strings",
          "example": [["issue"], ["ticket"], ["task"]]
        }
      ]
    },
    {
      "tool_description": "Summarizes a list of objects. The logic of how to summarize a particular object type is an internal implementation detail.",
      "tool_name": "summarize_objects",
      "title": "summarize_objects",
      "return_type": "array of objects",
      "argument_list": [
        {
          "argument_name": "objects",
          "argument_description": "List of objects to summarize",
          "argument_type": "array of objects",
          "example": [["issue1"], ["task1", "issue3"]]
        }
      ]
    },
    {
      "tool_description": "Returns a list of objects sorted by priority.",
      "tool_name": "prioritize_objects",
      "title": "prioritize_objects",
      "return_type": "array of objects",
      "argument_list": [
        {
          "argument_name": "objects",
          "argument_description": "A list of objects to be prioritized",
          "argument_type": "array of objects",
          "example": [["issue4"], ["task2", "issue3"], ["ticket9"]]
        }
      ]
    },
    {
      "tool_description": "Adds the given work items to the sprint",
      "tool_name": "add_work_items_to_sprint",
      "title": "add_work_items_to_sprint",
      "return_type": "none",
      "argument_list": [
        {
          "argument_name": "work_ids",
          "argument_description": "A list of work item IDs to be added to the sprint.",
          "argument_type": "array of strings",
          "example": [["deve/0:issue/6"], ["devdon:core:dvrv-us-1:task/1"]]
        },
        {
          "argument_name": "sprint_id",
          "argument_description": "The ID of the sprint to which the work items should be added.",
          "argument_type": "str",
          "example": ["sprint_4", "sprint_1"]
        }
      ]
    },
    {
      "tool_description": "Given a search string, returns the ID of a matching object in the system of record. If multiple matches are found, it returns the one where the confidence is highest.",
      "tool_name": "search_object_by_name",
      "title": "search_object_by_name",
      "return_type": "string",
      "argument_list": [
        {
          "argument_name": "query",
          "argument_description": "The search string, for example, customer's name, part name, user name.",
          "argument_type": "string",
          "example": ["DEV-123", "REV-432"]
        }
      ]
    },
    {
      "tool_description": "Returns the ID of the current sprint.",
      "tool_name": "get_sprint_id",
      "title": "get_sprint_id",
      "return_type": "string",
      "argument_list": []
    },
    {
      "tool_description": "Given a text, extracts actionable insights, and creates tasks for them, which are kind of a work item.",
      "tool_name": "create_actionable_tasks_from_text",
      "title": "create_actionable_tasks_from_text",
      "return_type": "array of strings",
      "argument_list": [
        {
          "argument_name": "text",
          "argument_description": "The text from which the actionable insights need to be created.",
          "argument_type": "string",
          "example": [
            "Transcript from slack channels",
            "Transcripts from a meeting",
            "workplace report"
          ]
        }
      ]
    },
    {
      "tool_description": "Returns the ID of the current user.",
      "tool_name": "whoami",
      "title": "whoami",
      "return_type": "string",
      "argument_list": []
    },
    {
      "tool_description": "Returns a list of work items that are similar to the given work item",
      "tool_name": "get_similar_work_items",
      "title": "get_similar_work_items",
      "return_type": "array of objects",
      "argument_list": [
        {
          "argument_name": "work_id",
          "argument_description": "The ID of the work item for which you want to find similar items",
          "argument_type": "string",
          "example": ["der/0:issue/2", "ton:core:dvrv-us-3:sprint/10"]
        }
      ]
    }
  ]
}


### Functions

In [ ]:
API_URL = "https://rjmy54al17scvxjr.us-east-1.aws.endpoints.huggingface.cloud"
headers = {
        "Content-Type": "application/json"
}
def query(payload):
	"""
	Sends a payload to a TGI endpoint.
	"""
	import requests
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

def query_raven(prompt):
	"""
	This function sends a request to the TGI endpoint to get Raven's function call.
	This will not generate Raven's justification and reasoning for the call, to save on latency.
	"""
	import requests
	output = query({
		"inputs": prompt,
		"parameters" : {"temperature" : 0.001, "stop" : ["<bot_end>"], "do_sample" : False, "max_new_tokens" : 2000}})
	call = output[0]["generated_text"].replace("Call:", "").strip()
	return call

def query_raven_with_reasoning(prompt):
	"""
	This function sends a request to the TGI endpoint to get Raven's function call AND justification for the call
	"""
	import requests
	output = query({
		"inputs": prompt,
		"parameters" : {"temperature" : 0.001, "do_sample" : False, "max_new_tokens" : 2000}})
	call = output[0]["generated_text"].replace("Call:", "").strip()
	return call

In [ ]:
import ast
import json

def generate_prompt(query, tools, instruction = None):
  '''
  Function:
  def {tool_name}({arg_name}):
      """
      {tool_desc}

      Args:
      {arg_name} (arg_dtype): {arg_desc}

      Returns:
      {return_dtype}: {tool_desc}
      """

  User Query: {query}<human_end>

  '''
  out = ""
  for tool in tools:
    out += f"""
Function:
def {tool['tool_name']}("""
    for arg in tool["argument_list"]:
      out += f"{arg['argument_name'].replace('.', '__dot__')}, "
    if len(tool["argument_list"]):
      out = out[:-2]
    out += f'''):\n\t"""\n\t{tool["tool_description"]}\n'''
    if len(tool["argument_list"]):
      out+='''\n\tArgs: \n'''
    for arg in tool["argument_list"]:
      out += f"\t{arg['argument_name'].replace('.', '__dot__')} ({arg['argument_type']}): {arg['argument_description']}\n"
    out += f"\n\tReturns:\n\t{tool['return_type']}: {tool['tool_description']}"
    out+= '\n\t"""\n\n'
  if instruction:
    out+= f"INSTRUCTION: {instruction}\n\n"
  out+= f"User Query: {query}<human_end>"
  return out

def parse_function_calls(call):
  """Parses a nested function call into a JSON object.

  Args:
    call: The function call to parse.

  Returns:
    A JSON object representing the function call.
  """

  # Parse the function call using AST.
  ast_tree = ast.parse(call)

  # Get the function name.
  if hasattr(ast_tree, "body"):
    function_name = ast_tree.body[0].value.func.id
  # Get the function arguments.
    function_arguments = []
    for argument in ast_tree.body[0].value.keywords:
      if isinstance(argument.value, ast.Call):
        function_arguments.append({argument.arg:parse_function_calls(argument.value)})
      else:
        function_arguments.append(
            {argument.arg: argument.value.value}
        )
  else:
    function_name = ast_tree.func.id
    function_arguments = []
    for argument in ast_tree.keywords:
      if isinstance(argument.value, ast.Call):
        function_arguments.append({argument.arg:parse_function_calls(argument.value)})
      else:
        function_arguments.append(
            {argument.arg: ast.literal_eval(argument.value)}
        )
  # Create a JSON object representing the function call.
  return {
      "function_name": function_name,
      "function_arguments": function_arguments
  }

def convert(call):
  ans = []
  funcs = []
  json_object = parse_function_calls(call)
  funcs.append(json_object)
  i = 0
  while i < len(funcs):
    args = []
    for arg in funcs[i]["function_arguments"]:
      arg_n = list(arg.keys())[0]
      if type(list(arg.values())[0]) == dict:
        if list(arg.values())[0].get("function_name"):
          funcs.append(list(arg.values())[0])
          arg_v = f"$$PREV_{list(arg.values())[0].get('function_name')}"
          args.append({
            arg_n.replace("__dot__", ".") : arg_v
          })
        else:
          for key in list(arg.values())[0]:
            args.append({
            f"{arg_n}.{key}" : list(arg.values())[0][key]
            })
      else:
        args.append({
            arg_n.replace("__dot__", ".") : list(arg.values())[0]
            })

    ans.append({
        "tool_name" : funcs[i]["function_name"],
        "arguments" : args
    })
    i+=1
  ans = list(reversed(ans))
  tool_names = [a["tool_name"] for a in ans]
  for i, a in enumerate(ans):
    for j, arg in enumerate(a["arguments"]):
      if list(arg.values())[0].startswith('$$PREV'):
        k = tool_names.index(list(arg.values())[0][7:])
        ans[i]["arguments"][j][list(ans[i]["arguments"][j].keys())[0]] = f"$$PREV[{k}]"
  return ans

def raven(query, t = tools["tools"]):
  return convert(query_raven(generate_prompt(query, t)))

### Test

In [ ]:
q = "Given a customer meeting transcript T, create action items and add them to my current sprint"

raven_call = query_raven_with_reasoning(generate_prompt(q, tools["tools"]))
print(f"Raven's Reasoning: {raven_call}")

json_ans = raven(q)
print("JSON\n",json.dumps(json_ans, indent = 4))